In [1]:
# 모두 1 또는 0인 텐서
import tensorflow as tf
x = tf.ones(shape=(2,1))
x

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[1.],
       [1.]], dtype=float32)>

In [2]:
x = tf.zeros(shape=(2,1))
x

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

In [3]:
# 랜덤 텐서
x = tf.random.normal(shape=(3,1), mean=0, stddev=1.)
x

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.21667375],
       [-2.2618635 ],
       [-0.6545166 ]], dtype=float32)>

In [4]:
x = tf.random.uniform(shape=(3,1), minval=0, maxval=1.)
x

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.26768947],
       [0.80077565],
       [0.17020035]], dtype=float32)>

In [8]:
# 텐서플로 텐서에 값을 할당하지 못함
x = tf.ones(shape=(2,2))
x[0, 0] = 0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

```
튜플처럼 에러가 발생한다.
해결 방법은 이걸 변할수 있게 하는 Variable() 안에 넣어주면 변경 가능하게 해준다.
```

In [9]:
x = tf.Variable(x)
x

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [11]:
x[0,0].assign(10)   # 할당해주기

<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=float32, numpy=
array([[10.,  1.],
       [ 1.,  1.]], dtype=float32)>

In [12]:
x.assign_add(tf.ones((2,2)))

<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=float32, numpy=
array([[11.,  2.],
       [ 2.,  2.]], dtype=float32)>

In [15]:
# GradientTape
input_var = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    result = tf.square(input_var)
gradient = tape.gradient(result, input_var)

In [17]:
# 상수 텐서 입력과 Gradient Tape
input_const = tf.constant(3.) # constant는 상수라는 뜻
with tf.GradientTape() as tape:
    tape.watch(input_const)  # watch는 잘 감시하라는 것
    result = tf.square(input_const)
gradient = tape.gradient(result, input_const)

```
텐서플로는 기본적으로 훈련 가능한 변수만 추적한다.
상수 텐서의 경우 tape.watch()를 호출하여 추적한다는 것을 수동으로 알려줘야 한다.
```

In [23]:
# 중첩 Gradient Tape(이계 도함수)
time = tf.Variable(10.)  # time은 10
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position = 4.9 * time**2
    speed = inner_tape.gradient(position, time)  # speed = 4.9 * 2 * time
acceleration = outer_tape.gradient(speed, time)  # acceleration의 결과물은 9.8

In [20]:
speed

<tf.Tensor: shape=(), dtype=float32, numpy=98.0>

In [21]:
acceleration

<tf.Tensor: shape=(), dtype=float32, numpy=9.8>

In [22]:
time = tf.Variable(tf.ones((2,2)))
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position = 4.9 * time**2
    speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time) 

In [39]:
# Layer의 서브클래스로 구현한 Dense 층
from tensorflow import keras

class SimpleDense(keras.layers.Layer):  ## 새로 만드는 댄스 레이어는 위에서 레이어를 만들어놓은것을 그대로 상속받았다는것


    def __init__(self, units, activation=None):
        super(SimpleDense, self).__init__()   ##중요. 위에서 상속받을때 부모클래스인 Layer에서 init가 오버리아딩 되니까 위의 부모클래서 init를 불러오라는것

        self.units = units
        self.activation = activation

    def build(self, input_shape):  ## 여기서 가중치의 틀을 잡아준다.ihputshape가 들어옴
        input_dim = input_shape[-1]  ## -1은 맨 뒤어꺼만 똑 땐다.
        self.W = self.add_weight(shape=(input_dim, self.units),
                    ## add_weight( )는 가중치를 간편하게 만들 수 있는 메서드이다.
                    ## 이 메서드는 독립적으로 변수를 생성하고 층의 속성으로 할당할 수도 있다.
                                 initializer='random_normal')  ## 초기값 설정
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros')

    def call(self, inputs):  ## 처리하는 연산을 call을 통해 만들기
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [43]:
# 인스턴스가 마치 함수처럼 사용된다.

my_dense = SimpleDense(units=32, activation=tf.nn.relu)  ## activation은 위의 self.activation에 들어간다.
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)

output_tensor.shape

TensorShape([2, 32])

In [ ]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)

output_tensor.shape

손실함수 종류 3가지 정리해두기 p138
올바른 손실 함수 선택하기 p232

In [49]:
import numpy as np
x = np.arange(1, 31)
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [55]:
# 8:2로 나누고 섞기
np.random.shuffle(x)
num = int(len(x)*0.8)
train_data = x[:num]
train_val = x[num:]

In [56]:
train_data

array([ 3, 24,  6, 13, 26, 22,  2, 18, 23, 29, 20, 21, 27, 19,  8,  5,  9,
        4,  7, 14, 12,  1, 15, 10])

In [57]:
train_val

array([17, 28, 16, 11, 30, 25])

```
위에와 동일 방법
train_test_split(x, test_size=0.2)
```

In [59]:
# p299

In [80]:
pip install kaggle

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73049 sha256=2883abedad7830979807eb1d6bb10dad62ee7d8b91660336fb821d6d030695e2
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\ac\b2\c3\fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [85]:
import os, shutil, pathlib

In [86]:
original_dir = pathlib.Path('train')

In [87]:
new_base_dir = pathlib.Path('cats_vs_dogs_small')

In [92]:
def make_subset(subset_name, start_index, end_index):
    for category in ('cat', 'dog'):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f'{category}.{i}.jpg'
                 for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                           dst=dir / fname)

In [93]:
make_subset('train', start_index=0, end_index=1000)
make_subset('validation', start_index=1000, end_index=1500)
make_subset('test', start_index=1500, end_index=2500)

FileNotFoundError: [Errno 2] No such file or directory: 'train\\cat.1.jpg'

In [84]:
# 이미지를 훈련, 검증, 테스트 디렉터리로 복사하기
import os, shutil, pathlib

original_dir = pathlib.Path('train')
new_base_dir = pathlib.Path('cat_vs_dogs_small')
def make_subset(subset_name, start_index, end_index):
    for category in ('cat', 'dog'):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f'{category}.{i}.jpg'
                 for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                           dst=dir / fname)

make_subset('train', start_index=0, end_index=1000)
make_subset('validation', start_index=1000, end_index=1500)
make_subset('test', start_index=1500, end_index=2500)

FileNotFoundError: [Errno 2] No such file or directory: 'train\\cat.0.jpg'

In [63]:
# 강아지 vs 고양이 분류 모델 만들기
from tensorflow import keras
from keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [64]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [65]:
# 모델 훈련 설정하기
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [75]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / 'train',
    image_size=(180, 180),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
                        new_base_dir / 'validation',
                        image_size=(180, 180),
                        batch_size=32)

test_dataset = image_dataset_from_directory(
                new_base_dir / 'test',
                image_size=(180, 180),
                batch_size=32)

NameError: name 'new_base_dir' is not defined

In [66]:
history = model.fit(train_data, epochs=30, validation_data=validation_data, callbacks=callbacks)

NameError: name 'validation_data' is not defined